In [2]:
import pandas as pd 

v1 = pd.read_csv("data/cost-of-living.csv", index_col=0)
v2 = pd.read_csv("data/cost-of-living_v2.csv")

In [3]:
# Exploring differences between dataset versions
outer = pd.merge(v1, v2, how='outer', indicator=True)
outer.groupby("_merge")['city'].count() # 


_merge
left_only     4030
right_only    4112
both           844
Name: city, dtype: int64

In [4]:
# At least some of these differences are due to different values across the quantitative columns:
outer.sort_values("city").head(2)

,city,country,x1,x2,x3,x4,x5,x6,x7,x8,...,x48,x49,x50,x51,x52,x53,x54,x55,data_quality,_merge
6833,'s-Hertogenbosch,Netherlands,18.97,63.22,9.11,4.35,3.16,3.06,2.83,2.29,...,964.18,753.43,1830.88,1448.90,4531.1,3301.74,2669.49,2.27,1,right_only
2147,'s-Hertogenbosch,Netherlands,18.70,62.32,8.98,4.28,3.12,3.02,2.79,2.26,...,950.41,742.67,1804.74,1428.21,4466.4,3254.58,2631.36,2.27,1,left_only


In [5]:
# Some of these differences are from cities being included in one version but not the other
city_diff = pd.merge(v1.city, v2.city, how = "outer", indicator=True) 
city_diff = city_diff[city_diff._merge != "both"]

dropped_cities = list(city_diff[city_diff['_merge'] == "left_only"].city) # Cities present in v1 but not v2
v1_only = v1.loc[v1.city.isin(dropped_cities)]

In [102]:
cost_living = pd.merge(v1_only, v2, how = "outer")

# The below file is from text copy-pasted from kaggle page's description section:
      # https://www.kaggle.com/datasets/mvieira101/global-cost-of-livin
column_descriptions = pd.read_csv("data/column_descriptions.csv",sep="	") 
column_descriptions = column_descriptions.iloc[1:56,] # Drop descriptions of columns whose names we don't want to change
col_key = dict(zip(column_descriptions['city'],column_descriptions['Name of the city']))
cost_living.rename(col_key, axis=1, inplace = True)
cost_living.head(1)
# The original dummy columns were entirely uninformative, but now many of these are too long. Alas.

,city,country,"Meal, Inexpensive Restaurant (USD)","Meal for 2 People, Mid-range Restaurant, Three-course (USD)",McMeal at McDonalds (or Equivalent Combo Meal) (USD),"Domestic Beer (0.5 liter draught, in restaurants) (USD)","Imported Beer (0.33 liter bottle, in restaurants) (USD)","Cappuccino (regular, in restaurants) (USD)","Coke/Pepsi (0.33 liter bottle, in restaurants) (USD)","Water (0.33 liter bottle, in restaurants) (USD)",...,1 Pair of Men Leather Business Shoes (USD),Apartment (1 bedroom) in City Centre (USD),Apartment (1 bedroom) Outside of Centre (USD),Apartment (3 bedrooms) in City Centre (USD),Apartment (3 bedrooms) Outside of Centre (USD),Price per Square Meter to Buy Apartment in City Centre (USD),Price per Square Meter to Buy Apartment Outside of Centre (USD),Average Monthly Net Salary (After Tax) (USD),"Mortgage Interest Rate in Percentages (%), Yearly, for 20 Years Fixed-Rate",data_quality
0,Guigang,China,2.3,20.95,4.4,NaN,2.79,2.1,0.49,0.3,...,130.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.18,0


#### Cleaning the Data Set

In [98]:
cost_living.drop_duplicates() # No duplicates...beautiful


,city,country,"Meal, Inexpensive Restaurant (USD)","Meal for 2 People, Mid-range Restaurant, Three-course (USD)",McMeal at McDonalds (or Equivalent Combo Meal) (USD),"Domestic Beer (0.5 liter draught, in restaurants) (USD)","Imported Beer (0.33 liter bottle, in restaurants) (USD)","Cappuccino (regular, in restaurants) (USD)","Coke/Pepsi (0.33 liter bottle, in restaurants) (USD)","Water (0.33 liter bottle, in restaurants) (USD)",...,1 Pair of Men Leather Business Shoes (USD),Apartment (1 bedroom) in City Centre (USD),Apartment (1 bedroom) Outside of Centre (USD),Apartment (3 bedrooms) in City Centre (USD),Apartment (3 bedrooms) Outside of Centre (USD),Price per Square Meter to Buy Apartment in City Centre (USD),Price per Square Meter to Buy Apartment Outside of Centre (USD),Average Monthly Net Salary (After Tax) (USD),"Mortgage Interest Rate in Percentages (%), Yearly, for 20 Years Fixed-Rate",data_quality
0,Guigang,China,2.30,20.95,4.40,NaN,2.79,2.10,0.49,0.30,...,130.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.18,0
1,Sidi Bouzid,Tunisia,1.85,16.22,2.78,1.39,1.31,0.69,0.50,0.20,...,59.74,154.49,154.49,370.77,370.77,NaN,NaN,253.36,8.20,0
2,Zaria,Nigeria,1.24,5.52,5.63,1.13,1.58,2.14,0.23,0.25,...,74.35,112.65,225.29,337.94,901.16,NaN,NaN,157.70,17.67,0
3,Paulista,Brazil,4.10,18.63,5.59,1.12,1.86,1.18,0.93,0.37,...,40.90,108.08,93.17,195.65,121.12,NaN,NaN,195.65,9.25,0
4,Houma,United States,10.00,NaN,10.00,NaN,NaN,5.00,NaN,NaN,...,89.50,950.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4965,Peterborough,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4966,Georgetown,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4967,Ixtapa Zihuatanejo,Mexico,5.16,30.94,12.89,0.98,NaN,1.80,0.62,0.41,...,103.14,412.55,257.84,515.69,412.55,NaN,NaN,NaN,NaN,0
4968,Iqaluit,Canada,29.65,74.27,13.71,6.67,8.89,3.71,3.52,4.08,...,NaN,NaN,NaN,2964.60,2964.60,NaN,NaN,NaN,6.53,0


In [112]:
cost_living.apply(lambda col: col.isna().sum(), axis=0).sort_values(ascending=False)

Tennis Court Rent (1 Hour on Weekend) (USD)                                       2410
Price per Square Meter to Buy Apartment Outside of Centre (USD)                   2315
Price per Square Meter to Buy Apartment in City Centre (USD)                      2237
Monthly Pass (Regular Price) (USD)                                                2177
International Primary School, Yearly for 1 Child (USD)                            1687
Apartment (3 bedrooms) Outside of Centre (USD)                                    1519
One-way Ticket (Local Transport) (USD)                                            1514
Apartment (3 bedrooms) in City Centre (USD)                                       1488
Average Monthly Net Salary (After Tax) (USD)                                      1439
Apartment (1 bedroom) Outside of Centre (USD)                                     1436
Taxi 1hour Waiting (Normal Tariff) (USD)                                          1401
Fitness Club, Monthly Fee for 1 Adult (USD)